# 00 - Environment and Conventions

This notebook initializes the environment, defines conventions, and runs a smoke test comparing Black-Scholes and Monte Carlo pricing.

In [ ]:
# Colab setup
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    get_ipython().system('pip install -q numpy pandas scipy matplotlib ipywidgets pytest')
    # Optional data package
    get_ipython().system('pip install -q yfinance')
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    print('Local runtime detected; skip Colab-only installation cell.')


In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd
import matplotlib
import scipy

from src.config import config_dict
from src.utils import ensure_dirs, set_seed
from src.black_scholes import bs_call_price
from src.monte_carlo import mc_price_european_gbm_terminal

cfg = config_dict(fast_mode=True)
set_seed(cfg['SEED'])
ensure_dirs([
    str(ROOT / cfg['RESULTS_DIR']),
    str(ROOT / cfg['FIG_DIR']),
    str(ROOT / cfg['TABLE_DIR']),
    str(ROOT / 'results' / 'logs'),
    str(ROOT / 'results' / 'reports'),
])

print('numpy', np.__version__)
print('pandas', pd.__version__)
print('scipy', scipy.__version__)
print('matplotlib', matplotlib.__version__)
print('Conventions: continuous compounding, time in years, dt=T/n_steps')


In [ ]:
bs = bs_call_price(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
mc = mc_price_european_gbm_terminal(
    S0=cfg['S0'],
    K=cfg['K'],
    r=cfg['R'],
    q=cfg['Q'],
    sigma=cfg['SIGMA'],
    T=cfg['T'],
    n_paths=20_000,
    option_type='call',
    antithetic=True,
    seed=cfg['SEED'],
)

smoke = pd.DataFrame([
    {
        'bs_call_price': bs,
        'mc_price': mc['price'],
        'ci_low': mc['ci_low'],
        'ci_high': mc['ci_high'],
        'bs_inside_ci': mc['ci_low'] <= bs <= mc['ci_high'],
    }
])
out_path = ROOT / 'results' / 'tables' / 'smoke_test.csv'
smoke.to_csv(out_path, index=False)
smoke
